# Semana 3 Laboratorio 1:<br>Implementando DataOps con Terraform

En este laboratorio, utilizarás [Terraform](https://www.terraform.io/intro), como una
herramienta de infraestructura como código (IaC), para desplegar una instancia de base de datos a través de un host bastión.

Para abrir los archivos de la solución, sigue estos pasos:

- Ve al menú principal y selecciona `Archivo -> Preferencias -> Configuración`.
- Haz clic en `editor de texto` a la izquierda, luego desplázate hacia abajo hasta la sección `Archivos: Excluir`.
- Elimina la línea `**/terraform_solution/**`. La carpeta ahora aparecerá en el explorador.
- Puedes cerrar la pestaña `Configuración`.

# Tabla de Contenidos
- [ 1 - Introducción al Laboratorio](#1)
- [ 2 - Configuración de Infraestructura](#2)
  - [ 2.1 - Recursos Proporcionados](#2-1)
  - [ 2.2 - Backend](#2-2)
  - [ 2.3 - Variables](#2-3)
  - [ 2.4 - Redes](#2-4)
  - [ 2.5 - RDS](#2-5)
  - [ 2.6 - Host Bastión](#2-6)
  - [ 2.7 - Salidas de Terraform](#2-7)
- [ 3 - Despliegue de Infraestructura](#3)
  - [ 3.1 - Inicialización](#3-1)
  - [ 3.2 - Planificación](#3-2)
  - [ 3.3 - Despliegue de Infraestructura](#3-3)
- [ 4 - Conexión a la Base de Datos](#4)
  - [ 4.1 - Asegurarse de que tu RDS es Privado](#4-1)
  - [ 4.2 - Conectándose al Host Bastión](#4-2)
  - [ 4.3 - Creando una Tabla](#4-3)
  - [ 4.4 - Poblando la tabla e inspeccionando los datos](#4-4)
- [ 5 - Desmantelamiento de tu Infraestructura](#5)


<div id='1'/>

## 1 - Introducción al Laboratorio

Un host de salto, también conocido como servidor de salto, es un servidor intermediario que actúa como una puerta de enlace o un puente que conecta a los usuarios autorizados desde Internet público con los recursos alojados dentro de una red privada. Cuando un usuario autorizado desea acceder a la red privada, debe establecer una conexión SSH o shell seguro con el host de salto para acceder de manera segura a los recursos privados. Usando una clave SSH, los usuarios externos pueden demostrar su identidad para conectarse al host de salto. Una vez que los usuarios están autenticados, el host de salto puede reenviar sus solicitudes a la red interna. Al hacer cumplir las políticas de seguridad, un host de salto puede mitigar el riesgo de acceso no autorizado a sistemas sensibles dentro de la red.

![Arquitectura](./images/bastion_host.drawio.png)

Aquí está el diagrama de la arquitectura del host de salto que implementarás en este laboratorio. Consiste en una instancia de base de datos RDS alojada dentro de una subred privada de un VPC dado, y una instancia EC2 que actúa como el host de salto. La instancia EC2 reside en una subred pública dentro del mismo VPC para que pueda recibir tráfico externo desde Internet público antes de retransmitir el tráfico seguro a la base de datos interna. En este laboratorio, el VPC con sus subredes privadas y públicas ya están creados y te son proporcionados. Utilizarás Terraform para crear la instancia de base de datos y la instancia EC2 que está actuando como el host de salto.

También usarás Terraform para generar un par de claves SSH, que consiste en una clave pública que almacenarás en la instancia EC2 y una clave privada correspondiente que guardarás en un archivo separado. El par de claves públicas y privadas se utilizan para cifrar y descifrar los mensajes entre el usuario externo y el host de salto durante una conexión SSH, para que la conexión externa pueda demostrar su identidad. Una vez que todos los recursos estén creados usando Terraform, la clave privada puede usarse para conectarse a la base de datos RDS a través del host de salto.

### Archivos de Configuración de Terraform

La carpeta `terraform` contiene los archivos de configuración de Terraform que editarás y ejecutarás para aprovisionar los recursos para tu arquitectura de host de salto. Los archivos están organizados de la siguiente manera:

- Dentro de la carpeta modules, puedes encontrar el módulo `bastion_host` que consiste en los archivos de configuración para cada una de las instancias RDS y EC2 así como los recursos de red (`rds.tf`, `ec2.tf`, `network.tf`). El módulo también incluye roles y políticas IAM (`iam_roles.tf` y `policies.tf`) y otros archivos de configuración que contienen las definiciones de las variables de entrada del módulo, valores de salida y proveedores requeridos (`variables.tf`, `outputs.tf`, `providers.tf`).
- Fuera de la carpeta modules, puedes encontrar los archivos principales de configuración de terraform del directorio raíz (`main.tf`, `variables.tf`, `outputs.tf`, `backend.tf`). El `main.tf` declara el módulo `bastion_host` y las variables de entrada del módulo. También puedes encontrar el archivo `tfvars` que usarás para asignar valores a algunas variables de entrada.

Cuando ejecutes Terraform para ejecutar comandos o aplicar cambios, Terraform concatenará todos estos archivos `*.tf` en un solo archivo de configuración.

> **IMPORTANTE**
> Los permisos de AWS que se te han otorgado están configurados con el [principio de menor privilegio](https://csrc.nist.gov/glossary/term/least_privilege#:~:text=Definitions%3A,needs%20to%20perform%20its%20function.) 
y generalmente están restringidos por nombre (por ejemplo, los recursos deben tener un nombre con el prefijo `de-c2w3lab1`) o por especificaciones del recurso (por ejemplo, usando un tipo de instancia `t3.small`). Por favor, sigue las instrucciones y abstente de crear otros recursos.


<div id='2'/>

## 2 - Configuración de Infraestructura

En esta sección, editarás múltiples archivos de configuración. Asegúrate de leer
las instrucciones cuidadosamente y de guardar tus cambios en cualquiera de los archivos presionando `Ctrl+S` o `Cmd+S`.


<div id='2-1'/>

### 2.1 - Recursos proporcionados

Se le proporcionan los recursos de red para su infraestructura, como
el VPC y sus subredes privadas y públicas. Estos recursos fueron creados a través de
una pila de CloudFormation.

Para acceder a la consola de AWS, ejecute el siguiente comando en la terminal.
Si es necesario, abra una nueva terminal seleccionando Terminal > Nueva Terminal desde el menú.

```bash
cat ../.aws/aws_console_url
```

Abra el enlace en la nueva ventana del navegador.

*Nota*: Por razones de seguridad, la URL para acceder a la consola de AWS expirará cada 15 minutos.
Por favor, vuelva a ejecutar el comando para obtener un nuevo enlace activo.

*Nota:* Si ve la ventana como en la siguiente captura de pantalla, haga clic en el enlace **logout**,
cierre la ventana y haga clic en el enlace de la consola nuevamente.

![AWSLogout](images/AWSLogout.png)

Busque **CloudFormation** en la barra de búsqueda. Haga clic en el ID alfanumérico de la pila
y luego haga clic en la pestaña **Outputs**. Si desplaza hacia abajo, puede encontrar los
nombres de cada uno de los recursos mencionados anteriormente. El valor de la columna contiene el ID
para cada recurso, que luego utilizará en los archivos de terraform.


<div id='2-2'/>

### 2.2 - Backend

Cuando aplicas una configuración de Terraform, Terraform genera un archivo `tfstate`.
Este archivo especifica el estado de la configuración - mostrando qué recursos
han sido creados. Puedes compartir el archivo `tfstate` con equipos que trabajan en la
misma colección de recursos usando el servicio [Terraform
Cloud](https://cloud.hashicorp.com/products/terraform) y servicios de AWS,
y puedes protegerlo de ediciones simultáneas.

Completa las líneas del bloque [backend local](https://www.terraform.io/language/settings/backends/s3) en el archivo [`backend.tf`](terraform/backend.tf) con el ID de la Cuenta de AWS. El bloque de backend local
define la ruta local, donde se almacenará el `tfstate`. La ruta local usa
el ID de la Cuenta de AWS asignado a ti. Para obtener este número, ve a la consola de AWS y haz clic
en la parte superior derecha, donde aparece tu nombre de usuario. Copia el ID de la Cuenta
haciendo clic en el botón de copiar junto a él. Reemplaza el marcador de posición con el ID de la Cuenta que copiaste.

![AWSLogout](images/AWSAccountID.png)

*Nota*: Recuerda guardar tus cambios en cualquiera de los archivos presionando `Ctrl+S`
o `Cmd+S`.


<div id='2-3'/>

### 2.3 - Variables

Se le proporcionan dos archivos `variables.tf`:

- el archivo principal `variables.tf`
  ([terraform/variables.tf](terraform/variables.tf)): contiene las
  definiciones de las variables utilizadas en el archivo `main.tf`. Estas variables son
  utilizadas para pasar valores a las variables de entrada del módulo `bastion_host`.
- el archivo `variables.tf` en el módulo `bastion_host`
  ([terraform/modules/bastion_host/variables.tf](terraform/modules/bastion_host/variables.tf)):
  contiene las definiciones de las variables utilizadas en los archivos de configuración
  dentro del módulo.

Para cada archivo, primero lea todas las variables definidas y luego complete las líneas
necesarias (indicadas entre los comentarios `START CODE HERE` y `END CODE HERE`).
Asegúrese de leer los comentarios proporcionados para cada bloque de variables.

Las variables que representan los IDs del VPC y sus subredes no tienen
valores predeterminados. Para inyectar valores para estas variables, modifique el
archivo `terraform.tfvars`
([terraform/terraform.tfvars](terraform/terraform.tfvars)). Utilice la información
de la pestaña **Outputs** (columna **Value**) en CloudFormation para completar estos
valores.

*Nota*:

- En general, no debe subir al repositorio el archivo `terraform.tfvars`
  si contiene datos sensibles.
- Recuerde guardar sus cambios en cualquiera de los archivos presionando `Ctrl+S` o
  `Cmd+S`.


<div id='2-4'/>

### 2.4 - Redes

En el archivo `network.tf`  
([terraform/modules/bastion_host/network.tf](terraform/modules/bastion_host/network.tf)),  
puedes encontrar los recursos y las fuentes de datos asociados con la gestión de la red, como la VPC, subredes y grupos de seguridad.

- Dado que la VPC y sus subredes ya están creadas y proporcionadas para ti,  
se declaran como [Bloques de Datos](https://www.terraform.io/language/data-sources) en  
el archivo `network.tf`. El bloque de datos que declara la VPC ya está definido,  
pero los bloques de datos para las subredes correspondientes están incompletos.  
Completa el código para crear bloques de datos de tipo  
[`aws_subnet`](https://registry.terraform.io/providers/hashicorp/aws/latest/docs/data-sources/subnet)  
para cada una de las subredes públicas y privadas de la VPC. Para definir el argumento `id` en cada bloque de datos, asegúrate de usar las variables que representan el  
ID de cada subred desde el archivo [variables.tf](terraform/modules/bastion_host/variables.tf) del módulo `bastion_host`.

- Lee los dos bloques restantes en el archivo  
  [`network.tf`](terraform/modules/bastion_host/network.tf). Estos bloques  
  declaran dos recursos de tipo  
  [`aws_security_group`](https://registry.terraform.io/providers/hashicorp/aws/latest/docs/resources/security_group)  
  que definen los grupos de seguridad que se adjuntarán a la base de datos RDS y  
  al host bastión.

Utilizarás estos recursos de red en los archivos de configuración de las instancias RDS y EC2 (host bastión).

*Nota*: Recuerda guardar tus cambios en cualquiera de los archivos presionando `Ctrl+S`  
o `Cmd+S`.


<div id='2-5'/>

### 2.5 - RDS

En el archivo `rds.tf`  
([terraform/modules/bastion_host/rds.tf](terraform/modules/bastion_host/rds.tf)),  
configurarás los recursos necesarios para el despliegue de tu instancia RDS.

- El primer bloque genera una contraseña aleatoria que usarás como la  
  contraseña maestra para la instancia de la base de datos.

- El segundo bloque crea un  
[`aws_db_subnet_group`](https://registry.terraform.io/providers/hashicorp/aws/latest/docs/resources/db_subnet_group)  
recurso para la instancia de base de datos RDS. Este grupo consiste en dos subredes  
(según la documentación de AWS, al menos dos subredes deben ser especificadas cuando  
se crea una instancia de base de datos RDS, en caso de que posteriormente quieras cambiar a  
despliegue en varias zonas de disponibilidad). Completa este bloque para crear el grupo de subredes usando las dos subredes privadas que declaraste como bloques de datos en el  
archivo [`network.tf`](terraform/modules/bastion_host/network.tf).

- El tercer bloque crea un  
  [`aws_db_instance`](https://registry.terraform.io/providers/figma/aws-4-49-0/latest/docs/resources/db_instance)  
  recurso:  
  - Para `instance_class`, usa el tipo de instancia `db.t3.micro`.  
  - Para `db_subnet_group_name`, usa el nombre del grupo de subredes que creaste en el segundo bloque.  
  - Para `vpc_security_group_ids`, usa el grupo de seguridad que creaste para la instancia RDS en el archivo `network.tf`.  
  - Para `username`, usa la variable de nombre de usuario maestra definida en el  
    [archivo variables.tf](terraform/modules/bastion_host/variables.tf) en el  
    módulo `bastion_host`.  
  - Para `password`, usa la contraseña maestra generada en el primer bloque.

*Nota*: Recuerda guardar tus cambios en cualquiera de los archivos presionando `Ctrl+S` o `Cmd+S`.


<div id='2-6'/>

### 2.6 - Bastion Host

En el archivo `ec2.tf`  
([terraform/modules/bastion_host/ec2.tf](terraform/modules/bastion_host/ec2.tf)),  
configurarás los recursos necesarios para el despliegue de tu bastion host.

- El primer bloque declara un [`tls_private_key`  
  recurso](https://registry.terraform.io/providers/hashicorp/tls/latest/docs/resources/private_key)  
  que generará un par de claves SSH. El segundo bloque almacena la clave privada en  
  un archivo local.  
- El cuarto bloque crea un  
  [`aws_key_pair`](https://registry.terraform.io/providers/hashicorp/aws/latest/docs/resources/key_pair)  
  recurso que puedes usar para registrar la clave pública SSH con la instancia EC2. Completa este bloque para especificar la clave pública usando la  
  [`tls_private_key`  
  recurso](https://registry.terraform.io/providers/hashicorp/tls/latest/docs/resources/private_key)  
  que declaraste en el primer bloque.  
- El último bloque crea un  
  [`aws_instance`](https://registry.terraform.io/providers/hashicorp/aws/latest/docs/resources/instance)  
  recurso o en otras palabras, la instancia EC2 que representa tu bastion host:  
  - Para `instance_type`, usa el tipo de instancia `t3.nano`.  
  - Para `key_name`, usa el `key_name` del recurso `aws_key_pair` que creaste en el bloque anterior.  
  - Para `subnet_id`, usa la subred pública en la zona de disponibilidad solicitada  
    (A) que creaste en el archivo  
    [`network.tf`](terraform/modules/bastion_host/network.tf).  
  - Para `vpc_security_group_ids`, usa el grupo de seguridad que creaste para el  
    bastion host en el archivo `network.tf`.  
  - Para `iam_instance_profile`, ya se ha definido un perfil de instancia IAM en el archivo `iam_roles.tf`.  

*Nota*: Recuerda guardar tus cambios en cualquiera de los archivos presionando `Ctrl+S` o `Cmd+S`.


<div id='2-7'/>

### 2.7 - Salidas de Terraform

Se le proporcionan dos archivos `outputs.tf`:

- En el archivo `outputs.tf` del host bastión
  ([terraform/modules/bastion_host/outputs.tf](terraform/modules/bastion_host/outputs.tf)),
  primero revise todos los valores de salida del archivo y luego complete los bloques necesarios
  para definir salidas para:
  - ID de la instancia del host bastión: refiérase al recurso `aws_instance` que creó
    en [`ec2.tf`](terraform/modules/bastion_host/ec2.tf).
  - DNS público del host bastión: también refiérase al recurso `aws_instance` que
    creó en [`ec2.tf`](terraform/modules/bastion_host/ec2.tf) y busque
    el atributo dns público
    [aquí](https://registry.terraform.io/providers/hashicorp/aws/latest/docs/resources/instance#attribute-reference).
  - Host de RDS (o dirección): refiérase al recurso `aws_db_instance` que creó
    en `rds.tf` y busque el atributo host/dirección
    [aquí](https://registry.terraform.io/providers/hashicorp/aws/latest/docs/resources/db_instance#attribute-reference).
  - Puerto de RDS: mismo comentario anterior.
  - Nombre de usuario de la base de datos: mismo comentario anterior.
  - Contraseña de la base de datos. Tenga en cuenta que esta es una salida sensible. Use `password` para reemplazar el tercer `None`.
- En el directorio raíz, el archivo `outputs.tf`
  ([terraform/outputs.tf](terraform/outputs.tf)) exporta los valores de salida del
  módulo `bastion_host` para que pueda usarlos cuando ejecute sus archivos terraform. No necesita editar nada en este archivo, solo asegúrese de revisarlo.

Con esta información de salida, podrá conectarse a la instancia del host bastión
y conectarse de forma segura a su base de datos privada.

*Nota*: Recuerde guardar sus cambios en cualquiera de los archivos presionando `Ctrl+S` o `Cmd+S`.


<div id='3'/>

## 3 - Implementación de infraestructura

Ahora que tus archivos de configuración de Terraform están listos, es momento de desplegar la
infraestructura. Para ello, verás que el proceso se puede dividir en unas
cuantas etapas.


<div id='3-1'/>

### 3.1 - Inicialización

`terraform init` es el primer paso para usar Terraform con una configuración nueva o existente. Cuando ejecutas `terraform init`, Terraform lee los archivos de configuración en el directorio (los que tienen la extensión `*.tf`) y inicializa el directorio de trabajo. Durante la inicialización, Terraform descarga los proveedores necesarios (especificados en el archivo [`providers.tf`](terraform/modules/bastion_host/providers.tf)) requeridos para la configuración.

En tu terminal, cambia tu directorio de trabajo a la carpeta `terraform`:

```bash
cd terraform
```

y luego, ejecuta el comando de inicialización:

```bash
terraform init
```


<div id='3-2'/>

#### 3.2 - Planificación

Después de la inicialización, puedes generar un plan de ejecución usando `terraform plan`.
Este paso compara el estado actual de la infraestructura con el estado deseado
descrito en los archivos de configuración. Terraform analiza los archivos de configuración,
verifica si hay errores de sintaxis y determina qué acciones son necesarias para
lograr el estado deseado. Luego, muestra un resumen de los cambios planeados,
incluyendo qué recursos serán creados, modificados o eliminados. Sin embargo,
`terraform plan` no realiza ningún cambio en tu infraestructura.

Ejecuta el siguiente comando para generar el plan de ejecución

```bash
terraform plan
```


<div id='3-3'/>

#### 3.3 - Implementación de infraestructura

Una vez que hayas revisado el plan y estés listo para hacer cambios, puedes aplicar los
cambios usando `terraform apply`. Este paso ejecuta las acciones descritas en el
plan de ejecución generado por el terraform plan. Terraform se comunica con las
APIs de los proveedores de la nube respectivos para crear, actualizar o eliminar recursos según
sea necesario y para alinear la infraestructura con el estado deseado descrito en
los archivos de configuración. Terraform también actualiza su archivo de estado
`terraform.tfstate` para reflejar el estado actual de la infraestructura después
de aplicar los cambios.

Ahora es el momento de desplegar tu infraestructura, en la terminal, ejecuta el
siguiente comando:

```bash
terraform apply
```

Verás una salida similar a la generada por el plan de ejecución y una
pantalla de confirmación. Escribe `yes` y verás el proceso de creación de cada
uno de los recursos. **Este proceso puede tomar un tiempo (alrededor de 7-10 minutos)** ya que
el despliegue de las instancias RDS y del host bastión puede tomar algún tiempo.

Al final del proceso, Terraform mostrará en la terminal las salidas que
definiste en el archivo [main `outputs.tf`](terraform/outputs.tf). Usarás
esta información en los siguientes pasos del laboratorio.

*Nota*: Si hay errores en los comandos o archivos de configuración de Terraform, la terminal puede fallar.
Cuando esto sucede, verás el siguiente mensaje:

![etl_diagram](images/terminal_crash.png)

Puedes volver a abrir la terminal presionando <code>Ctrl + \`</code> (o <code>Cmd + \`</code>) o navegando a Ver > Terminal.
En la terminal, vuelve a la carpeta de Terraform (`cd terraform`) y luego intenta
ejecutar nuevamente los comandos requeridos. El error ahora debería aparecer en la terminal.
Si la terminal continúa fallando, ejecuta el siguiente comando en su lugar:
`terraform apply -no-color  2> errors.txt`
Esto creará un archivo de texto que contiene el mensaje de error sin causar que la terminal falle.


<div id='4'/>

## 4 - Conexión a la Base de Datos


<div id='4-1'/>

### 4.1 - Asegurándose de que su RDS sea Privada

Una vez que la infraestructura ha sido desplegada con éxito, puede conectarse a la base de datos. Como primera prueba, intentará conectarse desde la terminal. Puede obtener el punto final de la base de datos desde la salida anterior o ejecutando el comando:

```bash
terraform output db_host
```

Este comando mostrará la contraseña (todas las salidas están entre comillas dobles):
```bash
terraform output db_master_password
```

Ahora conéctese a la base de datos reemplazando `<RDS-HOST>` con el valor de `db_host`:

```bash
psql -h <RDS-HOST> -U postgres_admin -p 5432 -d postgres --password
```

Puede ver que la conexión no se realiza. Su RDS ha sido configurada para ser privada y la única forma de acceder a ella es a través del host bastión.

Para salir del intento, presione `Ctrl+C` (también podría ser `Cmd+C` o `Ctrl+S`) o espere a que aparezca el error de tiempo de espera.


<div id='4-2'/>

### 4.2 - Conectándose al Host Bastión

Para conectarse al host bastión, hay varias opciones como:

* Conectarse directamente al host bastión usando SSH.
* Crear una conexión de túnel SSH al host bastión desde su entorno de desarrollo.

Para el propósito de este laboratorio, utilizará EC2 Instance Connect para interactuar con el host Bastión. Cambie a la pestaña de la consola de AWS y busque **EC2**. En el panel izquierdo, haga clic en **Instances**. Busque su instancia de host bastión `de-c2w3lab1-bastion-host` y haga clic en su ID. Luego, haga clic en **Connect**.

![Connect_to_instance](./images/Connect_to_instance.png)

Se le solicitará la terminal para interactuar con su host bastión. Con la terminal abierta, puede acceder a RDS usando este comando:

```bash
psql -h <RDS-HOST> -U postgres_admin -p 5432 -d postgres --password
```

Se le pedirá que escriba la contraseña de la base de datos.


<div id='4-3'/>

### 4.3 - Creando una Tabla

Ahora que puedes conectarte a la base de datos alojada en una instancia privada de RDS, puedes comenzar a trabajar con ella. Primero, crearás una tabla. Hemos proporcionado algunos archivos sql y datos en el host bastión, el primer script `sql/ratings_table_ddl.sql` crea la tabla de prueba.

```bash
\i sql/ratings_table_ddl.sql
```


<div id='4-4'/>

### 4.4 - Poblando la tabla e inspeccionando los datos

Los datos para poblar la tabla que creaste se encuentran en
`data/ratings_ml_training_dataset.csv` mientras que el script para poblarla está en
`sql/copy_data.sql`. En la misma terminal del host bastión, ejecuta el siguiente comando:

```bash
\i sql/copy_data.sql
```

Después de este proceso, deberías poder consultar tu tabla. Como punto de partida
para explorar tus datos, puedes usar la siguiente consulta y también intentar contar el
número de filas que han sido insertadas.

```sql
SELECT * FROM ratings_training LIMIT 10;
```

¡Genial, has interactuado con una base de datos alojada en un RDS privado a través de un
host bastión que fue desplegado usando Terraform como una herramienta IaC!

Sal del prompt de `psql` con el comando `\q`.


<div id='5'/>

## 5 - Desmantelar tu Infraestructura

Otra ventaja importante de usar una herramienta de IaC para gestionar tu infraestructura
es que puedes destruir fácilmente algunos recursos o incluso desmantelar toda tu
infraestructura. El proceso de limpieza es más fácil de realizar cuando ya no vas a usar
una cierta infraestructura. Para hacerlo, verifica que estás en el
directorio `terraform` y ejecuta el siguiente comando:

```bash
terraform destroy
```  

Ejecuta el comando en la terminal para limpiar los recursos no deseados. Verás el plan de ejecución para el proceso de destrucción y se te pedirá
confirmación. Responde `sí` para realizar el proceso de limpieza.

**¡Felicidades!** En este laboratorio, has implementado con éxito los principios de DataOps
usando Terraform como una herramienta de Infraestructura como Código (IaC) para automatizar
el despliegue de una arquitectura de host de bastión.
